In [1]:
import pandas as pd
import glob2
import matplotlib.pyplot as pltd
import datetime
import re
from datetime import datetime, timedelta
%matplotlib inline

In [2]:
locale_name = "ng"
read_enc_val ="ISO-8859-1"
read_enc_val ="cp1252"
read_enc_val ="utf-8"

In [3]:
global_music_file_paths = glob2.glob('spotify ' + locale_name + ' v2/*.csv')

In [4]:
print(len(global_music_file_paths))

106


In [5]:
print(global_music_file_paths[:4])

['spotify ng v2\\regional-ng-weekly-2022-02-17.csv', 'spotify ng v2\\regional-ng-weekly-2022-02-24.csv', 'spotify ng v2\\regional-ng-weekly-2022-03-03.csv', 'spotify ng v2\\regional-ng-weekly-2022-03-10.csv']


In [6]:
# global_music_file_paths

#### Add Dates to the Files

In [8]:
#Add the dates to the files - run once
def addsDatesToData():
    to_remove_front = "spotify " +locale_name+ " v2\\regional-" +locale_name+"-weekly-"
    to_remove_back = ".csv"

    len_front_remove = len(to_remove_front)
    len_back_remove = len(to_remove_back)

    for file in global_music_file_paths: 
#         print(file)
        the_data_in_file = pd.read_csv(file)
#         the_data_in_file = the_data_in_file.iloc[1: , :]
#         the_data_in_file.columns = ["Position", "Track Name", "Artist", "Streams", "URL"]
        
        
        the_dates = file[len_front_remove:]
        the_dates = the_dates[:-len_back_remove]

        split_dates = the_dates.split("-")

        start_date = split_dates[0] + "-" + split_dates[1] + "-" + split_dates[2]
        

        len_dates = the_data_in_file.shape[0]
        start_dates = len_dates * [start_date]

        date_columns = {"End Date": start_dates}   

        the_data_in_file["End Date"] = start_dates

        #write the data back to the File
        the_data_in_file.to_csv(file, index=False)
        
#remove dates in existing files
def removeExistingDates():
    for file in global_music_file_paths:
        
        the_data_in_file = pd.read_csv(file)

        the_data_in_file.drop(["Start Date", "End Date"], axis=1, inplace=True) 

        the_data_in_file.to_csv(file, index=False)
    
#check for dates ... opting for removal and putting them back on


In [9]:
# removeExistingDates()
addsDatesToData()

In [10]:
#Combine the files and sort by dates
all_files = []
trouble_files = []
for file in global_music_file_paths:
    the_data_in_file = pd.read_csv(file)
    
    if("URL" in list(the_data_in_file.columns)):
    
        the_data_in_file = the_data_in_file.drop(["URL"], axis=1)
    
    list_vals = sum(list(the_data_in_file.isnull().sum()))
    
    if(list_vals > 0):
        trouble_files.append(file)
    all_files.append(the_data_in_file)
    
the_spotify_data = pd.concat(all_files) 
# trouble_files_df = pd.concat(trouble_files)

In [11]:
all_cols = list(the_spotify_data.columns)
drop_cols = []

for col in all_cols:
    if("Unnamed" in col or "artificial" in col):
        drop_cols.append(col)
        
the_spotify_data.drop(drop_cols, axis=1, inplace = True)

In [12]:
the_spotify_data.columns

Index(['rank', 'uri', 'artist_names', 'track_name', 'source', 'peak_rank',
       'previous_rank', 'weeks_on_chart', 'streams', 'End Date'],
      dtype='object')

In [13]:
# the_spotify_data

In [14]:
# the_spotify_data["End Date"].unique().tolist()

### Error checking that all the Thursday dates are present

In [16]:
date_list = the_spotify_data["End Date"].unique().tolist()
start_date = date_list[0]
end_date = date_list[len(date_list)-1]

In [17]:
len(date_list)

106

In [18]:
start_date, end_date

('2022-02-17', '2025-04-10')

In [19]:
def check_thursdays_in_range(date_list, start_date, end_date):
    # Convert start_date and end_date strings to datetime objects
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Initialize a set to store Thursdays in the range
    thursdays_in_range = set()
    
    # Iterate through the range of dates and add Thursdays to the set
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() == 3:  # Thursday
            thursdays_in_range.add(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    
    # Convert date_list to a set for efficient membership checking
    date_set = set(date_list)
    
    # Find missing Thursdays
    missing_thursdays = sorted(thursdays_in_range - date_set)
    print("thursdays_in_range: ", len(thursdays_in_range))
    print("date_set: ", len(date_set))
    
    if missing_thursdays:
        print("Missing Thursdays in the range:")
        for thursday in missing_thursdays:
            print(thursday)
        return False
    else:
        print("All Thursdays are present in the range.")
        return True

In [20]:
result = check_thursdays_in_range(date_list, start_date, end_date)
print("All Thursdays present:", result)

thursdays_in_range:  165
date_set:  106
Missing Thursdays in the range:
2022-11-24
2022-12-01
2022-12-08
2022-12-15
2023-04-20
2023-05-04
2023-05-18
2023-05-25
2023-06-01
2023-06-15
2023-06-22
2023-06-29
2023-07-06
2023-07-13
2023-07-20
2023-07-27
2023-08-03
2023-08-10
2023-08-17
2023-08-24
2023-08-31
2023-09-07
2023-09-14
2023-09-21
2023-09-28
2023-10-05
2023-10-12
2023-10-19
2023-10-26
2023-11-02
2023-11-09
2023-11-16
2023-11-23
2023-11-30
2023-12-07
2023-12-14
2023-12-21
2023-12-28
2024-01-04
2024-01-11
2024-01-18
2024-01-25
2024-02-01
2024-02-08
2024-02-15
2024-02-22
2024-02-29
2024-03-07
2024-03-14
2024-03-21
2024-03-28
2024-04-04
2024-04-11
2024-04-18
2024-04-25
2024-05-02
2024-05-09
2024-06-06
2024-06-13
All Thursdays present: False


### Continue

In [22]:
the_spotify_data["Week"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["Year"] = the_spotify_data["Week"]

the_spotify_data['Week'] = the_spotify_data['Week'].apply(lambda x : x.isocalendar()[1] )
the_spotify_data['Year'] = the_spotify_data['Year'].apply(lambda x : x.isocalendar()[0] )

# the_spotify_data["Start Date Dt"] = the_spotify_data["Start Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["End Date Dt"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data['Seconds since Epoch'] = the_spotify_data['End Date Dt'].apply(lambda x : round(x.timestamp(), 0))

In [23]:
#get the month when this happened
def changedaytotheFirst(x):
    x = x[:-2] + "01"
    
    return x

In [24]:
the_spotify_data["End Date Aggregated"] = the_spotify_data["End Date"].apply(lambda x : changedaytotheFirst(x) )

In [25]:
#find the number of artists on the track
the_spotify_data.dropna(inplace=True)

the_spotify_data["artist_names"] = the_spotify_data["artist_names"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["artist_names"] = the_spotify_data["artist_names"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["artist_names"] = the_spotify_data.apply(lambda x : x["artist_names"] + "Artist_NA" if(x["artist_names"] == "") else x["artist_names"], axis=1)

the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["track_name"] = the_spotify_data.apply(lambda x : x["artist_names"] + "_NA" if(x["track_name"] == "") else x["track_name"], axis=1)

def countNumberArtists(x):
    len_all_featured_artists = 0
    num_artists = 0
    xvals = x.split("feat")
    len_xvals = len(xvals)
    
    if(len_xvals > 1):
        featured_artists = xvals[1:]
        featured_artists_str = ' '.join(featured_artists)
        all_featured_artists = featured_artists_str.split("& ")
        len_all_featured_artists = len(all_featured_artists)
        
    num_artists = 1 + len_all_featured_artists
    
    return num_artists
        
    
the_spotify_data['ArtistCount'] = the_spotify_data['track_name'].apply(lambda x : countNumberArtists(x))

In [26]:
the_spotify_data.shape

(21200, 16)

In [27]:
the_spotify_data["main_artist"] = the_spotify_data.apply(lambda x : x["artist_names"].split(",")[0], axis=1)

In [28]:
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["index"], axis=1, inplace=True)

In [29]:
the_spotify_data.shape

(21200, 17)

### Is Top Ten Status

In [31]:
def everInTopTen(x):
    position_vals = list(x["rank"])
    position_vals = list(set(position_vals))
    
#     print(position_vals)
    
    min_pos = min(position_vals)
    existence_val = 0
    
    if(min_pos <= 10):
        existence_val = 1
    
    x["isTopTen"] = [existence_val] * len(x)
    
    return x
    
the_spotify_data = the_spotify_data[~the_spotify_data.index.duplicated()]
the_spotify_data = the_spotify_data.groupby(["artist_names", "track_name"]).apply(lambda x : everInTopTen(x))

C:\Users\lkhum\AppData\Local\Temp\ipykernel_24728\3443158733.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data = the_spotify_data.groupby(["artist_names", "track_name"]).apply(lambda x : everInTopTen(x))


In [32]:
the_spotify_data.head()

rank                                   uri  \
artist_names track_name                                                    
1da Banton   Ego        7043    44  spotify:track:6XdAvTC80727JAI9HBAVLc   
                        7259    60  spotify:track:6XdAvTC80727JAI9HBAVLc   
                        7511   112  spotify:track:6XdAvTC80727JAI9HBAVLc   
                        7751   152  spotify:track:6XdAvTC80727JAI9HBAVLc   
                        7981   182  spotify:track:6XdAvTC80727JAI9HBAVLc   

                             artist_names track_name                   source  \
artist_names track_name                                                         
1da Banton   Ego        7043   1da Banton        Ego  Squareball Ent. Limited   
                        7259   1da Banton        Ego  Squareball Ent. Limited   
                        7511   1da Banton        Ego  Squareball Ent. Limited   
                        7751   1da Banton        Ego  Squareball Ent. Limited   
                        7981   1da Banton        Ego  Squareball Ent. Limited   

                              peak_rank  previous_rank  weeks_on_chart  \
artist_names track_name                                                  
1da Banton   Ego        7043         44             -1               1   
                        7259         44             44               2   
                        7511         44             60               3   
                        7751         44            112               4   
                        7981         44            152               5   

                              streams    End Date  Week  Year End Date Dt  \
artist_names track_name                                                     
1da Banton   Ego        7043    57614  2022-10-20    42  2022  2022-10-20   
                        7259    44738  2022-10-27    43  2022  2022-10-27   
                        7511    30477  2022-11-03    44  2022  2022-11-03   
                        7751    26225  2022-11-10    45  2022  2022-11-10   
                        7981    21051  2022-11-17    46  2022  2022-11-17   

                              Seconds since Epoch End Date Aggregated  \
artist_names track_name                                                 
1da Banton   Ego        7043         1.666224e+09          2022-10-01   
                        7259         1.666829e+09          2022-10-01   
                        7511         1.667434e+09          2022-11-01   
                        7751         1.668038e+09          2022-11-01   
                        7981         1.668643e+09          2022-11-01   

                              ArtistCount main_artist  isTopTen  
artist_names track_name                                          
1da Banton   Ego        7043            1  1da Banton         0  
                        7259            1  1da Banton         0  
                        7511            1  1da Banton         0  
                        7751            1  1da Banton         0  
                        7981            1  1da Banton         0

### When did the track reach the Top 10 

### how long does a song last on the charts?

In [35]:
the_spotify_data["isTopTen"].value_counts()

isTopTen
0    15180
1     6020
Name: count, dtype: int64

In [36]:
the_spotify_data.isnull().sum()

rank                   0
uri                    0
artist_names           0
track_name             0
source                 0
peak_rank              0
previous_rank          0
weeks_on_chart         0
streams                0
End Date               0
Week                   0
Year                   0
End Date Dt            0
Seconds since Epoch    0
End Date Aggregated    0
ArtistCount            0
main_artist            0
isTopTen               0
dtype: int64

In [37]:
unique_artists = the_spotify_data["main_artist"].unique().tolist()
num_artists = len(unique_artists)
print(num_artists)

493


### find new artists in that week

In [39]:
# the_spotify_data.head(12)
def getArtistAppearanceCount(x):

    x = x.sort_values(by='End Date', ascending=True)
    x = x.reset_index()
    # x.drop(["index"],axis=1,inplace=True)
    x = x.reset_index()
    
    x_cols = [w.replace('index', 'artistAppearanceCount') for w in x.columns]
    
    x.columns = x_cols
    
    return x        

the_spotify_data = the_spotify_data.drop(['artist_names', 'track_name'], axis=1)
the_spotify_data =the_spotify_data.groupby(["main_artist"], as_index=False).apply(lambda x : getArtistAppearanceCount(x))
the_spotify_data.drop(["level_2"], inplace=True, axis=1)

C:\Users\lkhum\AppData\Local\Temp\ipykernel_24728\3015101900.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data =the_spotify_data.groupby(["main_artist"], as_index=False).apply(lambda x : getArtistAppearanceCount(x))


In [40]:
the_spotify_data.head()

artistAppearanceCount                          artist_names  \
0 0                      0                            1da Banton   
  1                      1                            1da Banton   
  2                      2                            1da Banton   
  3                      3  1da Banton, Kizz Daniel, Tiwa Savage   
  4                      4                            1da Banton   

            track_name  rank                                   uri  \
0 0          No Wahala    36  spotify:track:4vHy2IHzf3EabEa7oMpUZB   
  1          No Wahala    33  spotify:track:4vHy2IHzf3EabEa7oMpUZB   
  2          No Wahala    46  spotify:track:4vHy2IHzf3EabEa7oMpUZB   
  3  No Wahala - Remix    56  spotify:track:4yaPPFIafaVaMRHZ5IHkbA   
  4          No Wahala    49  spotify:track:4vHy2IHzf3EabEa7oMpUZB   

                      source  peak_rank  previous_rank  weeks_on_chart  \
0 0  Squareball Ent. Limited         36             -1               1   
  1  Squareball Ent. Limited         33             36               2   
  2  Squareball Ent. Limited         33             33               3   
  3  Squareball Ent. Limited         56             -1               1   
  4  Squareball Ent. Limited         33             46               4   

     streams    End Date  Week  Year End Date Dt  Seconds since Epoch  \
0 0    41233  2022-02-17     7  2022  2022-02-17         1.645056e+09   
  1    41956  2022-02-24     8  2022  2022-02-24         1.645661e+09   
  2    39122  2022-03-03     9  2022  2022-03-03         1.646266e+09   
  3    30777  2022-03-10    10  2022  2022-03-10         1.646870e+09   
  4    36285  2022-03-10    10  2022  2022-03-10         1.646870e+09   

    End Date Aggregated  ArtistCount main_artist  isTopTen  
0 0          2022-02-01            1  1da Banton         0  
  1          2022-02-01            1  1da Banton         0  
  2          2022-03-01            1  1da Banton         0  
  3          2022-03-01            1  1da Banton         0  
  4          2022-03-01            1  1da Banton         0

#### Track Appearance Count

In [42]:
def getTrackAppearanceCount(x):

    x = x.sort_values(by=['End Date','track_name'], ascending=True)

    x = x.reset_index()
    # x.drop(["index"],axis=1,inplace=True)
    x = x.reset_index()
    
    x_cols = x.columns
    
    x_cols = [w.replace('index', 'trackAppearanceCount') for w in x_cols]
    
    x.columns = x_cols
    
    return x         
    
the_spotify_data =the_spotify_data.groupby(["artist_names", "track_name"], as_index=False).apply(lambda x : getTrackAppearanceCount(x))
# the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["level_0", "level_1"], inplace=True, axis=1)
# print(the_spotify_data.columns)
# stop

C:\Users\lkhum\AppData\Local\Temp\ipykernel_24728\2047836851.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data =the_spotify_data.groupby(["artist_names", "track_name"], as_index=False).apply(lambda x : getTrackAppearanceCount(x))


In [43]:
the_spotify_data.head()

trackAppearanceCount  artistAppearanceCount artist_names track_name  \
0 0                     0                     61   1da Banton        Ego   
  1                     1                     62   1da Banton        Ego   
  2                     2                     64   1da Banton        Ego   
  3                     3                     65   1da Banton        Ego   
  4                     4                     66   1da Banton        Ego   

     rank                                   uri                   source  \
0 0    44  spotify:track:6XdAvTC80727JAI9HBAVLc  Squareball Ent. Limited   
  1    60  spotify:track:6XdAvTC80727JAI9HBAVLc  Squareball Ent. Limited   
  2   112  spotify:track:6XdAvTC80727JAI9HBAVLc  Squareball Ent. Limited   
  3   152  spotify:track:6XdAvTC80727JAI9HBAVLc  Squareball Ent. Limited   
  4   182  spotify:track:6XdAvTC80727JAI9HBAVLc  Squareball Ent. Limited   

     peak_rank  previous_rank  weeks_on_chart  streams    End Date  Week  \
0 0         44             -1               1    57614  2022-10-20    42   
  1         44             44               2    44738  2022-10-27    43   
  2         44             60               3    30477  2022-11-03    44   
  3         44            112               4    26225  2022-11-10    45   
  4         44            152               5    21051  2022-11-17    46   

     Year End Date Dt  Seconds since Epoch End Date Aggregated  ArtistCount  \
0 0  2022  2022-10-20         1.666224e+09          2022-10-01            1   
  1  2022  2022-10-27         1.666829e+09          2022-10-01            1   
  2  2022  2022-11-03         1.667434e+09          2022-11-01            1   
  3  2022  2022-11-10         1.668038e+09          2022-11-01            1   
  4  2022  2022-11-17         1.668643e+09          2022-11-01            1   

    main_artist  isTopTen  
0 0  1da Banton         0  
  1  1da Banton         0  
  2  1da Banton         0  
  3  1da Banton         0  
  4  1da Banton         0

In [44]:
the_spotify_data.to_csv("the_spotify_data_" + locale_name+ ".csv")

In [45]:
the_spotify_data["rank"] = the_spotify_data["rank"].astype(int)
the_spotify_data["streams"] = the_spotify_data["streams"].astype(int)
the_spotify_data["ArtistCount"] = the_spotify_data["ArtistCount"].astype(int)

the_spotify_data["isTopTen"] = the_spotify_data["isTopTen"].astype(int)
# the_spotify_data["lenOnCharts"] = the_spotify_data["lenOnCharts"].astype(int)
the_spotify_data["Year"] = the_spotify_data["Year"].astype(int)
the_spotify_data["Week"] = the_spotify_data["Week"].astype(int)

In [46]:
all_cols = list(the_spotify_data.columns)
drop_cols = []

for col in all_cols:
    if("Unnamed" in col or "artificial" in col):
        drop_cols.append(col)
        
the_spotify_data.drop(drop_cols, axis=1, inplace = True)

In [47]:
the_spotify_data["Artist and Track"] = the_spotify_data["artist_names"] + "; " + the_spotify_data["track_name"]

### Getting the list of Artists and Tracks

In [49]:
import pandas as pd
# import pymongo
# import getlyrics

In [50]:
def trackAppearance(x):
#     x = x.reset_index()

    x = x.sort_values(by=["End Date"])
#     x.set_index(["Position"])

    x["rank"] = x["rank"].astype(int)
#     print(list(x["Position"]))
    
    x["rank difference"] = x["rank"].diff().fillna(0)
    x["rank difference"] = x["rank difference"].astype(int)
    
#     x["Position Difference"] = x["Position"].sub(x["Position"].shift())
#     x.drop(["index"], inplace=True)
    
    return x
    
all_files_grouped = the_spotify_data.groupby(["main_artist", "track_name"]).apply(lambda x: trackAppearance(x))

C:\Users\lkhum\AppData\Local\Temp\ipykernel_24728\2368203610.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_files_grouped = the_spotify_data.groupby(["main_artist", "track_name"]).apply(lambda x: trackAppearance(x))


In [51]:
all_files_grouped["rank difference"].max()

174

### Position over time

In [53]:
def positionvertime(x):
    posovertime = 0
    #new track
    if(x["rank difference"] == 0 and x["trackAppearanceCount"]==0):
        posovertime = 0
    #track stayed in the same position
    elif(x["rank difference"] == 0 and x["trackAppearanceCount"]!=0):
        posovertime = 50
    #track went up the chart
    elif(x["rank difference"] < 0):
        posovertime = 75
    #track fell down the chart
    else:
        posovertime = 100
    return posovertime

In [54]:
all_files_grouped["Position over Time"] = all_files_grouped.apply(lambda x : positionvertime(x), axis=1)

##### Add a Girl Group and Boy Group

In [56]:
ggdf = pd.read_csv("Classification/ListofGirlGroups.csv", encoding='ANSI')
bgdf = pd.read_csv("Classification/ListofBoyGroups.csv", on_bad_lines='skip', encoding='ANSI')

In [57]:
def IsGG(ggdf, y):
    ggs = [x.lower().strip() for x in ggdf["Artist"]]
    y = y.lower().strip()
    
    if y in ggs:
        return True
    else:
        return False

def IsBG(bgdf, y):
    bgs = [x.lower().strip() for x in bgdf["Artist"]]
    y = y.lower().strip()
    
    if y in bgs:
        return True
    else:
        return False

In [58]:
all_files_grouped["IsGirlGroup"] = all_files_grouped["main_artist"].apply(lambda x : IsGG(ggdf, x))
all_files_grouped["IsBoyGroup"] = all_files_grouped["main_artist"].apply(lambda x : IsGG(bgdf, x))

In [59]:
ggs = all_files_grouped[all_files_grouped["IsGirlGroup"]==True]["main_artist"].unique().tolist()
bgs = all_files_grouped[all_files_grouped["IsBoyGroup"]==True]["main_artist"].unique().tolist()

In [60]:
# all_files_grouped

In [61]:
all_files_grouped = all_files_grouped.droplevel(['main_artist', 'track_name'])

In [62]:
# all_files_grouped.drop(["level_2"], inplace=True, axis=1)

In [63]:
all_files_grouped.to_csv("Classification/all_files_"+locale_name+"v1.csv", index=False)

In [64]:
all_files_grouped.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'artist_names',
       'track_name', 'rank', 'uri', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [65]:
# new_all_files_grouped = all_files_grouped.reset_index(drop=True)

In [66]:
def newstuff(x):
#     x = x.reset_index()

    x = x.sort_values(by=['Seconds since Epoch'])
#     x.set_index(["Position"])
    print(x)
    
    return x
    


In [67]:
# all_files_grouped = new_all_files_grouped.groupby(["Artist", "Track Name"]).apply(lambda x: newstuff(x))

In [68]:
# all_files_grouped.min()

In [69]:
# df = pd.DataFrame({'A': 'a a b'.split(), 'B': [1,2,3], 'C': [4,6, 5]})

In [70]:
# new_all_files_grouped.groupby([])

In [71]:
# all_files_grouped.apply(lambda x : print(type(x)) )

In [72]:
# # new_all_files_grouped.h'Track Name', 'Artist'ead(5)
# cols_ = ['Start Date', 'Seconds since Epoch', 'Position']
# new_test_df = pd.DataFrame()

In [73]:
# 604800
def weekonweekposition(x):
    
#     print(x.values)
    
    pd.DataFrame({'email':x.index, 'list':x.values})
    
    
#     return x_
    
#     x_ = pd.DataFrame(x, columns=['Start Date'])
    
#     x_ = x.to_frame()
    
#     print(x_.columns)
    
#     for n in range(0, fin_len):
#         for col in cols_:
#             print(len(x[n]), x[n], col)
        
    
#     x_ = pd.DataFrame(x, columns=cols_)
#     print(x_.shape)
#     print(x.columns)

In [74]:
# all_files_gr = new_all_files_grouped[['Artist', 'Track Name', 'Seconds since Epoch', 'Position']]

In [75]:
# all_files_grouped[cols_].apply( lambda x : weekonweekposition(x) )

In [76]:
# new_test_df

### South African Artists

In [78]:
all_files_grouped.shape

(21200, 25)

In [79]:
all_files_grouped.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'artist_names',
       'track_name', 'rank', 'uri', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [80]:
narrowing_df = all_files_grouped[['End Date', "rank difference"]]

In [81]:
narrowing_df.reset_index(inplace=True)

In [82]:
# narrowing_df.drop(["level_2"], axis=1, inplace=True)

In [83]:
narrowing_df = narrowing_df[narrowing_df["rank difference"] == 0]

In [84]:
print("done.")

done.
